# Experiment 002: Submit the ORIGINAL Valid Baseline

This is the baseline from snapshot 21328309254 that scored 70.647327 on LB.
We submit it UNMODIFIED to establish a working LB score.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.ops import unary_union
from decimal import Decimal, getcontext
import json

getcontext().prec = 25

# Tree geometry
def get_tree_polygon():
    trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
    base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
    tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
    trunk_bottom_y = -trunk_h
    vertices = [
        (float(Decimal('0.0')), float(tip_y)),
        (float(top_w / Decimal('2')), float(tier_1_y)),
        (float(top_w / Decimal('4')), float(tier_1_y)),
        (float(mid_w / Decimal('2')), float(tier_2_y)),
        (float(mid_w / Decimal('4')), float(tier_2_y)),
        (float(base_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(base_y)),
        (float(-(base_w / Decimal('2'))), float(base_y)),
        (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
        (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
        (float(-(top_w / Decimal('4'))), float(tier_1_y)),
        (float(-(top_w / Decimal('2'))), float(tier_1_y)),
    ]
    return Polygon(vertices)

TREE_POLY = get_tree_polygon()
print(f"Tree: {len(TREE_POLY.exterior.coords)} vertices")

In [ ]:
def parse_s_value(s_val):
    if isinstance(s_val, str) and s_val.startswith('s'):
        return float(s_val[1:])
    return float(s_val)

def load_submission(path):
    df = pd.read_csv(path)
    df['x_val'] = df['x'].apply(parse_s_value)
    df['y_val'] = df['y'].apply(parse_s_value)
    df['deg_val'] = df['deg'].apply(parse_s_value)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    return df

def create_tree(x, y, deg):
    return translate(rotate(TREE_POLY, deg, origin=(0, 0)), x, y)

def get_bbox_side(polygons):
    if not polygons:
        return 0
    combined = unary_union(polygons)
    bounds = combined.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

# Load the ORIGINAL valid baseline
df = load_submission('/home/submission/submission.csv')
print(f"Loaded {len(df)} rows")

In [ ]:
# Compute score for all n
scores_by_n = {}

for n in range(1, 201):
    n_df = df[df['n'] == n]
    if len(n_df) != n:
        print(f"ERROR: n={n} has {len(n_df)} trees")
        continue
    polygons = [create_tree(row['x_val'], row['y_val'], row['deg_val']) for _, row in n_df.iterrows()]
    side = get_bbox_side(polygons)
    scores_by_n[n] = (side ** 2) / n

total_score = sum(scores_by_n.values())
print(f"\nTotal score: {total_score:.6f}")
print(f"Expected: 70.647327")
print(f"Target: 68.888293")
print(f"Gap: {total_score - 68.888293:.6f}")

In [ ]:
# Save metrics
metrics = {
    'cv_score': total_score,
    'expected_lb': 70.647327,
    'target': 68.888293,
    'gap': total_score - 68.888293,
    'source': 'snapshot_21328309254_unmodified'
}

with open('/home/code/experiments/002_valid_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Saved metrics: {metrics}")